In [25]:
from datetime import datetime

import pandas as pd

In [31]:
def date_parser(x):
    if str.lower(x) == 'unknown':
        return None
    return datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')

df = pd.read_csv('../data/fullsample.csv',
                 parse_dates=['BEGIN', 'END'],
                 date_parser=date_parser)


In [32]:
df = df.rename(columns=str.lower)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7395885 entries, 0 to 7395884
Data columns (total 12 columns):
 #   Column     Dtype         
---  ------     -----         
 0   jobid      object        
 1   state      object        
 2   begin      datetime64[ns]
 3   end        datetime64[ns]
 4   reqmem     object        
 5   usedmem    object        
 6   reqtime    object        
 7   usedtime   object        
 8   nodes      int64         
 9   cpus       int64         
 10  partition  object        
 11  exitcode   object        
dtypes: datetime64[ns](2), int64(2), object(8)
memory usage: 677.1+ MB


In [36]:
df['lifespan'] = df['end'] - df['begin']


In [37]:
df['lifespan']

0                      NaT
1         30 days 00:00:23
2         30 days 00:00:14
3         29 days 04:00:04
4         24 days 09:00:17
                ...       
7395880    0 days 00:01:46
7395881    0 days 00:01:25
7395882    0 days 00:03:14
7395883    0 days 00:01:56
7395884    0 days 00:03:08
Name: lifespan, Length: 7395885, dtype: timedelta64[ns]